In [129]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [130]:
data = pd.read_csv('data.csv')
alpha_data = pd.read_csv('alpha.csv', sep='\t')
data.head()

,6.16,a,4.19,a.1,3.12,a.2
0,200.91,4.2,155.34,8.80,180.64,11.95
1,33.00,3.6,30.80,8.50,50.53,10.55
2,27.18,3.2,28.66,8.35,36.66,9.80
3,28.47,2.8,25.35,7.95,42.69,9.35
4,18.86,2.3,26.30,7.40,38.52,8.90


In [131]:
alpha_data.head()

,a0,ai,ai2
0,0.00,-4.05,-3.98
1,0.00,-4.05,-4.00
2,0.05,-4.08,-4.02
3,-0.05,-4.05,-4.05
4,0.00,-4.00,-3.95


In [132]:
data1 = data.iloc[:, :2].dropna(axis=0, how='all')
data2 = data.iloc[:, 2:4].dropna(axis=0, how='all')
data3 = data.iloc[:, 4:].dropna(axis=0, how='all')
data_list = [data1, data2, data3]
data1.head()

,6.16,a
0,200.91,4.2
1,33.00,3.6
2,27.18,3.2
3,28.47,2.8
4,18.86,2.3


In [133]:
for d in data_list: 
    d.iloc[:, 0] = d.iloc[:, 0].cumsum()

data1.head()

,6.16,a
0,200.91,4.2
1,233.91,3.6
2,261.09,3.2
3,289.56,2.8
4,308.42,2.3


In [134]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import FuncFormatter

def format_x(value, tick_number):
    return f'{value:.0f}'

def format_y(value, tick_number):
    return f'{value:.2f}'

# 设置 matplotlib 参数
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.rm'] = 'Times New Roman'
plt.rcParams['mathtext.it'] = 'Times New Roman:italic'
plt.rcParams['mathtext.bf'] = 'Times New Roman:bold'
plt.rcParams['font.size'] = 16
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16


# 绘图
plt.figure(figsize=(8, 6), dpi=1200)

# 绘制散点图
plt.scatter(data1.iloc[:, 0], data1.iloc[:, 1], color='red', label=r'6.16 $\mathrm{~M}$ HCl')
plt.scatter(data2.iloc[:, 0], data2.iloc[:, 1], color='green', label=r'4.19 $\mathrm{~M}$ HCl')
plt.scatter(data3.iloc[:, 0], data3.iloc[:, 1], color='blue', label=r'3.12 $\mathrm{~M}$ HCl')

# 设置标签和图例
plt.xlabel(r'$t$ / s')
plt.ylabel(r'$\alpha_t$ / ${}^\circ$')
plt.legend(frameon=False)

# 设置网格线
plt.gca().xaxis.set_major_formatter(FuncFormatter(format_x))
plt.gca().yaxis.set_major_formatter(FuncFormatter(format_y))
plt.gca().xaxis.set_minor_locator(ticker.AutoMinorLocator())
plt.gca().yaxis.set_minor_locator(ticker.AutoMinorLocator())
plt.grid(which='major', alpha=0.5)
plt.grid(which='minor', alpha=0.2)

# 显示图形
plt.show()

In [135]:
alpha_0 = alpha_data.iloc[:, 0].mean()
alpha_infty = alpha_data.iloc[:, 1:].mean().mean()
alpha_0, alpha_infty

(0.0, -4.023)

In [136]:
d1_latex_list = []
d2_latex_list = []

for d in data_list:
    d['a-infty'] = d.iloc[:, 1] - alpha_infty
    d['lg a-infty'] = np.log10(d['a-infty'])

    # 设置保留的小数位数
    dd = d.copy()
    dd.iloc[:, 0] = dd.iloc[:, 0].map(lambda x: f'{x:.0f}')
    dd.iloc[:, 1] = dd.iloc[:, 1].map(lambda x: f'{x:.2f}')
    dd['a-infty'] = dd['a-infty'].map(lambda x: f'{x:.2f}')
    # 如果dd['a-infty']>10,dd['lg a-infty']保留四位小数，否则保留三位小数
    dd['lg a-infty'] = dd['lg a-infty'].map(lambda x: f'{x:.4f}' if float(x) > 1 else f'{x:.3f}')

    # 转换为 LaTeX
    ddd = dd.iloc[:, [0, 1, 1]]
    d1_latex = ddd.to_latex(index=False, escape=False)
    d1_latex_list.append(d1_latex)

    dddd = dd.iloc[:, [0, 2, 3]]
    d2_latex = dddd.to_latex(index=False, escape=False)
    d2_latex_list.append(d2_latex)

# 打印 LaTeX 表格
with open('data1_latex.txt', 'w') as f:
    for d_latex in d1_latex_list:
        f.write(d_latex)
        f.write('\n\n')

with open('data2_latex.txt', 'w') as f:
    for d_latex in d2_latex_list:
        f.write(d_latex)
        f.write('\n\n')


In [137]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import FuncFormatter
from scipy.optimize import curve_fit
from scipy.stats import linregress

def format_x(value, tick_number):
    return f'{value:.0f}'

def format_y(value, tick_number):
    return f'{value:.2f}'

# 设置 matplotlib 参数
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.rm'] = 'Times New Roman'
plt.rcParams['mathtext.it'] = 'Times New Roman:italic'
plt.rcParams['mathtext.bf'] = 'Times New Roman:bold'
plt.rcParams['font.size'] = 16
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16

# 线性拟合函数
def linear_model(x, a, b):
    return a * x + b

# 执行线性拟合并计算 R^2 和参数误差
def fit_and_calculate_r_squared(data, x_col, y_col):
    popt, pcov = curve_fit(linear_model, data[x_col], data[y_col])
    perr = np.sqrt(np.diag(pcov))  # 计算参数误差
    slope, intercept, r_value, _, _ = linregress(data[x_col], data[y_col])
    return popt, perr, r_value**2

def scientific_notation_with_error(value, error, significant_digits=2):
    value_str = f"{value:.{significant_digits}e}"
    error_str = f"{error:.{significant_digits}e}"
    value_num, value_exp = value_str.split('e')
    error_num, error_exp = error_str.split('e')

    error_decimal = error * 10 ** (-int(value_exp))

    if int(value_exp):
        return f"({value_num} \pm {error_decimal:.{significant_digits}f}) \\times 10^{{{int(value_exp)}}}"
    return f"({value_num} \pm {error_decimal:.{significant_digits}f})"

datasets = [data1, data2, data3]
labels = [r'6.16 $\mathrm{~M}$ HCl', r'4.19 $\mathrm{~M}$ HCl', r'3.12 $\mathrm{~M}$ HCl']
colors = [('darkred', 'red'), ('darkgreen', 'green'), ('darkblue', 'blue')]
kk_list = []

plt.figure(figsize=(8, 6), dpi=1200)

for data, label, color in zip(datasets, labels, colors):
    popt, perr, r_squared = fit_and_calculate_r_squared(data, data.columns[0], data.columns[-1])
    texa = scientific_notation_with_error(popt[0], perr[0])
    kk_list.append((popt[0], perr[0]))
    texb = scientific_notation_with_error(popt[1], perr[1])
    print(f"{label} Fit: y = {texa}x + {texb};\quad R^2 = {r_squared:.5f}")
    plt.scatter(data.iloc[:, 0], data.iloc[:, -1], color=color[0], label=label)
    plt.plot(data.iloc[:, 0], linear_model(data.iloc[:, 0], *popt), color=color[1])


# 设置标签和图例
plt.xlabel(r'$t$ / s')
plt.ylabel(r'$\lg\left[(\alpha_t-\alpha_\infty)/{}^\circ\right]$')
plt.legend(frameon=False)

# 设置网格线
plt.gca().xaxis.set_major_formatter(FuncFormatter(format_x))
plt.gca().yaxis.set_major_formatter(FuncFormatter(format_y))
plt.gca().xaxis.set_minor_locator(ticker.AutoMinorLocator())
plt.gca().yaxis.set_minor_locator(ticker.AutoMinorLocator())
plt.grid(which='major', alpha=0.5)
plt.grid(which='minor', alpha=0.2)

# 显示图形
plt.show()

6.16 $\mathrm{~M}$ HCl Fit: y = (-1.27 \pm 0.02) \times 10^{-3}x + (1.19 \pm 0.01);\quad R^2 = 0.99654
4.19 $\mathrm{~M}$ HCl Fit: y = (-5.68 \pm 0.11) \times 10^{-4}x + (1.21 \pm 0.01);\quad R^2 = 0.99154
3.12 $\mathrm{~M}$ HCl Fit: y = (-3.46 \pm 0.06) \times 10^{-4}x + (1.22 \pm 0.01);\quad R^2 = 0.99192


In [138]:
k_list = [(-i[0] * np.log(10), i[1] * np.log(10)) for i in kk_list]
k_list

[(0.0029228233893220366, 3.852235500407811e-05),
 (0.001307039424621774, 2.5745553655874614e-05),
 (0.0007964596383885846, 1.409382180188638e-05)]

In [139]:
# 要使得i_ in [np.log(2) / i[0] for i in k_list]与j_ in [np.log(2)*i[1]/i[0]**2 for i in k_list]一一对应

t05_list = [np.log(2) / i[0] for i in k_list] 
t05e_list = [np.log(2)*i[1]/i[0]**2 for i in k_list]

In [140]:
t05 = [(t, e) for t, e in zip(t05_list, t05e_list)]
t05

[(237.14986786140514, 3.125598157008831),
 (530.318494991477, 10.44600722082786),
 (870.2853818962334, 15.400211759692288)]

In [141]:
cH = [6.16, 4.19, 3.12]

In [142]:
latex_line_list = []

for c_, kk_, k_, t_ in zip(cH, kk_list, k_list, t05):
    c1 = f'{i:.2f}'
    c2 = scientific_notation_with_error(kk_[0], kk_[1])
    c3 = scientific_notation_with_error(k_[0], k_[1])
    c4 = scientific_notation_with_error(t_[0], t_[1])
    latex_line = f'{c_} & {c1} & ${c2}$ & ${c3}$ & ${c4}$ \\\\'
    latex_line_list.append(latex_line)

with open('latex_line.txt', 'w') as f:
    for line in latex_line_list:
        f.write(line)
        f.write('\n')

In [143]:
# 转化为latex 
k

15.400211759692288

In [147]:
cH = [6.16, 4.19, 3.12]
df_H = pd.DataFrame()
df_H['[\ce{H+}] / M'] = cH
df_H['\ln ([\ce{H+}] / M)'] = np.log(cH)
df_H['k/\si{M^{-1}}'] = [i[0] for i in k_list]
df_H['\ln (k/\si{M^{-1}})'] = np.log(df_H['k/\si{M^{-1}}'])

'''def float_to_sci(float_str, digits=2):
    float_str = float_str.split('e')
    float_str[0] = f"{float(float_str[0])}:.{digits}f"
    float_str_1 = 0 if float_str[1] else 

    if float_str_1:
        return float_str[0] + f" \\times 10^{{{float_str_1}}}"
    return float_str[0]

npdf_H = df_H.to_numpy()
H_line_list = []

for line in npdf_H:
    line = [float_to_sci(str(i)) for i in line]
    H_line_list.append(' & '.join(line) + ' \\\\')

with open('H_line.txt', 'w') as f:
    for line in H_line_list:
        f.write(line)
        f.write('\n')'''

IndexError: list index out of range

In [ ]:
# 转化为列表
data_H = df_H.iloc[:, [1, 3]].to_numpy().T


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import FuncFormatter
from scipy.optimize import curve_fit
from scipy.stats import linregress

def format_x(value, tick_number):
    return f'{value:.3f}'

def format_y(value, tick_number):
    return f'{value:.3f}'

# 设置 matplotlib 参数
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.rm'] = 'Times New Roman'
plt.rcParams['mathtext.it'] = 'Times New Roman:italic'
plt.rcParams['mathtext.bf'] = 'Times New Roman:bold'
plt.rcParams['font.size'] = 16
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16

# 线性拟合函数
def linear_model(x, a, b):
    return a * x + b

color = ('darkred', 'red')
kk_list = []

plt.figure(figsize=(8, 6), dpi=1200)

popt, perr, r_squared = fit_and_calculate_r_squared(data_H, 0, -1)
texa = scientific_notation_with_error(popt[0], perr[0], 2)
kk_list.append((popt[0], perr[0]))
texb = scientific_notation_with_error(popt[1], perr[1], 2)
print(f"{label} Fit: y = {texa}x + {texb};\quad R^2 = {r_squared:.5f}")
plt.scatter(data_H[0], data_H[1], color=color[0])
plt.plot(data_H[0], linear_model(data_H[0], *popt), color=color[1])

# 设置标签和图例
plt.xlabel(r'$\ln\left([\mathrm{H}^+]/\mathrm{M}\right)$')
plt.ylabel(r'$\ln\left(k/\mathrm{M}^{-1}\right)$')

# 设置网格线
plt.gca().xaxis.set_major_formatter(FuncFormatter(format_x))
plt.gca().yaxis.set_major_formatter(FuncFormatter(format_y))
plt.gca().xaxis.set_minor_locator(ticker.AutoMinorLocator())
plt.gca().yaxis.set_minor_locator(ticker.AutoMinorLocator())
plt.grid(which='major', alpha=0.5)
plt.grid(which='minor', alpha=0.2)

# 显示图形
plt.show()

3.12 $\mathrm{~M}$ HCl Fit: y = (1.92 \pm 0.12)x + (-9.35 \pm 0.17);\quad R^2 = 0.99642


In [ ]:
alpha_i = alpha_data.iloc[:, 1:].to_numpy().T.flatten()
alpha_i.mean(), alpha_i.std()

(-4.023, 0.037960505792204544)